### SEVER DEFINITIONS


In [1]:
# Necessary Imports
from flask import Flask, jsonify
from werkzeug.routing import IntegerConverter
from _csv import Dialect as _Dialect
import csv
import copy
import os
from rtls_slave import aoa_main, pixel_calculate


# Handle Negative Values
class SignedIntConverter(IntegerConverter):
    regex = r'-?\d+'
app = Flask(__name__)
app.url_map.converters['signed_int'] = SignedIntConverter
#Store input values: AoA and Horizontal Angle

aoabefore = {}
aoa= {}
lev = []

@app.route('/')
def home():
    return "This is the HomePage"

#Routing Settings
@app.route('/send/<signed_int:horiz>', methods=['GET'])
def get_angle(horiz):
    global aoa
    global aoabefore
    print("Plane Angle: " + str(horiz))
    aoa_result = aoa_main()
    input_letter = input("If you want to add this data to list type 'y', otherwise press Enter")
    if input_letter=='y':
        lev.append(horiz)
        aoabefore = copy.deepcopy(aoa)
        aoa = aoa_result
        
    return jsonify({'state': 'success'})

@app.route('/calculate/', methods=['GET'])
def calculate():
    print("Before Turning："+str(aoabefore)+'\n'+"After Turning： " + str(aoa) +'\n'+"Rotation Angle： "+ str(lev))
    #aoa_image()
    u = []
    v = []

    set_aoa = set(aoa)
    set_aoabefore = set(aoabefore)

    for slave in set_aoa.intersection(set_aoabefore):
        minnumber = min(len(aoabefore[slave]), len(aoa[slave]))
        f = open(f"./aoa_results/{slave.replace(':','_')}_aoa_data_before_calibration.csv",'a',newline="")
        f.truncate()
        writer = csv.writer(f)
        

        for i in range(minnumber):
            for j in range(minnumber):
                (temp1, temp2) = pixel_calculate(lev[-2] - lev[-1], aoabefore[slave][i], aoa[slave][j])

                if (temp1 != 1):
                    u.append(temp1)
                    v.append(temp2)
                    tupx = (temp1,temp2)
                    writer.writerow(tupx)
        
        f.close()

    
    return jsonify({'state': 'success'})

### RUNNING THE SEVER

In [2]:
app.run(host='0.0.0.0',port = 5000, debug=False) 

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://10.19.217.212:5000
Press CTRL+C to quit


Plane Angle: 122
Master : <RTLSNode(CC26x2 Master, started daemon 22456)> 
Passives : [] 
All : [<RTLSNode(CC26x2 Master, started daemon 22456)>]
Devices Reset
Start scan for 5 sec
Scan Results: [{'addr': '80:6F:B0:EE:9C:67', 'addrType': 0, 'rssi': -60, 'advSID': 1, 'periodicAdvInt': 240}, {'addr': 'F8:8A:5E:2D:89:EA', 'addrType': 0, 'rssi': -96, 'advSID': 255, 'periodicAdvInt': 0}, {'addr': 'F8:8A:5E:2D:89:EA', 'addrType': 0, 'rssi': -91, 'advSID': 1, 'periodicAdvInt': 240}, {'addr': '80:6F:B0:EE:9C:67', 'addrType': 0, 'rssi': -66, 'advSID': 255, 'periodicAdvInt': 0}]
Try to connect to : 80:6F:B0:EE:9C:67
Connected to : 80:6F:B0:EE:9C:67 with connection handle 0
Try to connect to : F8:8A:5E:2D:89:EA
Connected to : F8:8A:5E:2D:89:EA with connection handle 1
AOA Params Set for Connection Handle: 0
AOA Params Set for Connection Handle: 1
AOA Callback Set for All Connection Handlers
AOA Started with Connection Handle: 0
AOA Started with Connection Handle: 1
Going to sleep for 10 sec
Try t

10.19.190.195 - - [18/Mar/2023 12:15:43] "GET /send/122 HTTP/1.1" 200 -


Plane Angle: 122
Master : <RTLSNode(CC26x2 Master, started daemon 17096)> 
Passives : [] 
All : [<RTLSNode(CC26x2 Master, started daemon 17096)>]
Devices Reset
Start scan for 5 sec
Scan Results: [{'addr': 'F8:8A:5E:2D:89:EA', 'addrType': 0, 'rssi': -94, 'advSID': 255, 'periodicAdvInt': 0}, {'addr': '80:6F:B0:EE:9C:67', 'addrType': 0, 'rssi': -63, 'advSID': 255, 'periodicAdvInt': 0}]
AOA Callback Set for All Connection Handlers
Going to sleep for 10 sec
Try to stop AOA result parsing threadSTOP Command Received

AOA Stopped
Done
{}


...

Scan Results: [{'addr': '80:6F:B0:EE:9C:67', 'addrType': 0, 'rssi': -56, 'advSID': 255, 'periodicAdvInt': 0}, {'addr': 'F8:8A:5E:2D:89:EA', 'addrType': 0, 'rssi': -96, 'advSID': 1, 'periodicAdvInt': 240}, {'addr': 'F8:8A:5E:2D:89:EA', 'addrType': 0, 'rssi': -96, 'advSID': 255, 'periodicAdvInt': 0}, {'addr': '80:6F:B0:EE:9C:67', 'addrType': 0, 'rssi': -66, 'advSID': 1, 'periodicAdvInt': 240}]
Try to connect to : F8:8A:5E:2D:89:EA
Plane Angle: 98
Connected to : F8:8A:5E:2D:89:EA with connection handle 0
Try to connect to : 80:6F:B0:EE:9C:67
=== ERROR: No one of the nodes identified as RTLS MASTER ===
Try to stop AOA result parsing thread
AOA Stopped
Done
{}
Connected to : 80:6F:B0:EE:9C:67 with connection handle 1
AOA Params Set for Connection Handle: 0
AOA Params Set for Connection Handle: 1
AOA Callback Set for All Connection Handlers
AOA Started with Connection Handle: 0
AOA Started with Connection Handle: 1
Going to sleep for 10 sec


10.19.190.195 - - [18/Mar/2023 12:18:13] "GET /send/98 HTTP/1.1" 200 -


Try to stop AOA result parsing threadSTOP Command Received

AOA Stopped
Master Disconnected
Done
{'F8:8A:5E:2D:89:EA': [5, 5, 4, 4, 9, 14, 14, 12, 11, 12, 6, 6, 3, 6, 4, 6, 5, 5, 5, 5, 4, 5, 6, 5, 10, 7, 10], '80:6F:B0:EE:9C:67': [57, 58, 59, 58, 58, 57, 56, 55, 57, 56, 57, 57, 55, 56, 56, 57, 57, 57, 57, 56, 57, 56, 57, 59]}


10.19.190.195 - - [18/Mar/2023 12:18:28] "GET /send/98 HTTP/1.1" 200 -
f:\intern_VisBLE\VisBLE_INTERN\AoA Point Cloud\rtls_slave.py:254: RuntimeWarning: invalid value encountered in arccos
  azimuth1 = np.arccos(cosAOAangle1/sinelvation1)
10.19.190.195 - - [18/Mar/2023 12:18:33] "GET /calculate/ HTTP/1.1" 200 -


Before Turning：{'80:6F:B0:EE:9C:67': [21, 32, 35, 28, 29, 31, 28, 22, 19, 27, 25, 21, 28, 41, 38, 35, 37, 43, 45, 43, 25, 34, 31, 29], 'F8:8A:5E:2D:89:EA': [-4, -7, -9, -10, -12, -10, -12, -12, -12, -11, -10, -12, -13, -12, -11, -13, -4, -4, -1, -3]}
After Turning： {'F8:8A:5E:2D:89:EA': [5, 5, 4, 4, 9, 14, 14, 12, 11, 12, 6, 6, 3, 6, 4, 6, 5, 5, 5, 5, 4, 5, 6, 5, 10, 7, 10], '80:6F:B0:EE:9C:67': [57, 58, 59, 58, 58, 57, 56, 55, 57, 56, 57, 57, 55, 56, 56, 57, 57, 57, 57, 56, 57, 56, 57, 59]}
Rotation Angle： [122, 98]
